In [1]:
import pandas as pd

In [2]:
trips = pd.read_csv("T:/DCProjects/StoryMap/BikeCounting/BikeShare/Data/trips_org_dst.csv")

In [3]:
trips.head()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,PathID,Distance,Minutes,OriginDestination
0,10551719,19607,1197171,13th & Kincaid,44.045877,-123.078595,2019-01-01,00:30,"13th & Kincaid - 13th & Pearl, Pearl side",0.71,14.683333,Origin
1,10551732,19812,1404147,13th & Kincaid,44.045760,-123.078600,2019-01-01,00:34,"13th & Kincaid - 13th & Pearl, Pearl side",0.74,11.833333,Origin
2,10551790,19749,1210268,Matthew Knight Arena,44.045472,-123.065293,2019-01-01,01:07,Matthew Knight Arena - 13th & Kincaid,0.68,8.316667,Origin
3,10551796,19743,1210268,Matthew Knight Arena,44.045427,-123.065345,2019-01-01,01:10,Matthew Knight Arena - 13th & Kincaid,0.67,5.533333,Origin
4,10551856,19597,557417,Broadway & Willamette,44.049645,-123.092835,2019-01-01,01:50,Broadway & Willamette - Matthew Knight Arena,1.68,16.866667,Origin


In [4]:
excludedIDs = [717565, 742339, 764038, 819845, 1228447, 1354709, 1897910, 2184703, 2207685]

In [5]:
trips = trips[~trips.UserID.isin(excludedIDs)]

In [6]:
trips.Location = trips.Location.str.replace(', ', ' @ ', regex=True)

In [7]:
trips.Location = trips.Location.str.replace('Monroe St & Blair Blvd ', 'Monroe St & Blair Blvd', regex=True)

In [8]:
import geopandas as gpd

In [9]:
stations = gpd.read_file(r"T:\DCProjects\StoryMap\BikeCounting\BikeMap\data\mapdata\bike_share_stations.shp")

In [10]:
stations.head()

,station id,name,geometry
0,hub_3828,Eugene Family YMCA,POINT (-13701591.712 5471253.204)
1,hub_3829,Monroe St & Blair Blvd,POINT (-13704001.270 5473411.225)
2,hub_3830,10th & Olive SE,POINT (-13702770.365 5472975.425)
3,hub_3831,16th & Willamette,POINT (-13702642.423 5471973.384)
4,hub_3832,13th & Pearl @ Pearl side,POINT (-13702278.750 5472449.473)


In [11]:
"Monroe St & Blair Blvd" in stations.name.values

True

In [12]:
tripLocations = [x for x in trips.Location.unique() if x != 'nan']

In [13]:
tripLocSelected = []

In [14]:
Loc2Review = []

In [15]:
# check if the trip location names are matched with the station names
for loc in tripLocations:
    if loc in stations.name.values:
        tripLocSelected.append(loc)
    else:
        print(loc)
        Loc2Review.append(loc)

East 20th Avenue 600
Peace Health Rides
East 20th Avenue 601
Smith Family Books
South Eugene High School
Eugene-Springfield
East 19th Alley 600
Hillyard Alley
East 20th Avenue 608
East 20th Avenue 698
East 20th Avenue 638
20 E 13th Ave
1390 High St
1571 Alder St
944 1/2 E 19th Ave
1015 Pearl St
1202 Willamette St
Living Learning Center North
1437 Hilyard St
840 Willamette St
E/S of Olive S of 10th
998 Ferry Ln
199 W 8th Ave
1850 Oak St
1479 Moss St
1333 Onyx St
470 E 11th Ave
1215 E 13th Ave
1480 Mill St
210 Oak Alley
879 E 13th Ave
941 Willamette St
365 E 13th Ave
65 E 19th Alley
UO Student Recreation Center
1313 Pearl St
1341 Lincoln Alley
207 W 10th Ave
808 Pearl St
200-298 Ferry St
Prince Lucien Campbell Hall
W/S of Hilyard N of 26th
180 E 18th Ave
Charnel Mulligan City Park
1848 Orchard St
210 Cheshire Ave
520 Willamette St #10503
6168 Ferry St
1888 Franklin Blvd
328 E Broadway
101 E Broadway Suite 100
2055 Patterson St
765 E 17th Ave
650 W 12th Ave
W/S of Pearl N of 8th
Safeway
1

1391 E 21st Ave
564 E 13th Ave
1791 Alder St
33 E Broadway
66 W 10th Ave
297 E 5th Ave
1931 Madison St
Owen Cherry Tree
45 E 8th Ave
980 Willamette St Suite 200
1240 E 18th Ave
W/S of Pearl S of 17th
744 E 21st Ave
2191 Onyx St
2700 Hilyard St
2160 W 11th Ave D
2130 W 11th Ave
199 E 5th Ave #25
1376 Olive St
1870 E 15th Ave
1550 Walnut St
W/S of Patterson S of 13th
2250 Patterson St #166
63 W 29th Ave
2690 Willamette St
7 E 7th Ave
710 Tyler St
31 Oakway Center
200 W Broadway
777 Pearl St # 105
99-29 Shelton Mcmurphey Blvd
1600 Oak St
2009 Elk Ave
1037 E 19th Ave
1733 Pearl St D
967 Patterson St
1945 Kincaid St
1629 E 24th Ave
490 E Broadway
755 Jefferson St
824 E 28th Ave
205 W 24th Ave
Sladden Park
1280 Patterson St
University Health Center
3225 Kinsrow Ave
1871 Hilyard St
460 Coburg Rd Suite 306
73 Oak St
1733 Pearl St
1883 Mill St
1560 Villard St
1857 Franklin Blvd
W/S of Willamette S of 12th
E/S of Olive S of 12th
1873 E 15th Ave
1259 Willamette St
1814 Ferry Alley
51 McClure Ln
7

1900 Taylor St
966 E 18th Ave
E/S of High N of 4th
110 W 6th Ave
81 E 28th Ave
1912 Moss St
1643 Oak St
1880 Franklin Blvd
2455 W 23rd Ave
744 W Park St
1306 E 18th Ave
399 E 10th Ave
2090 Alder St
1877 Ferry St
1623 E 23rd Ave
1629 Moss St
722 Country Club Rd
1192 Jefferson St
110 W 7th Ave
Lane Transit District Customer Service Center
279 E 14th Ave
2450 Birch Ln
425 E 13th Ave
School of Education & Counseling
1555 Jefferson St
199 W 6th Ave
942 Fairway Pl
680 E 24th Ave
765 Clark St
1162 Charnelton St
2288 University St
360 W 27th Pl
651 W 7th Ave
853 Raintree Way
711 W 22nd Ave
515 High St
450 W 3rd Ave
1000 W 1st Ave
1237 Charnelton St
588 W 7th Ave
1120 E 24th Ave
454 Willamette St #204
1374 Willamette St
290 Marche Chase Dr
777 W 6th Ave
Greenway Bike Bridge
455 W 1st Ave
576 E. 15th Alley
1561 Ferry St
405 E 8th Ave # 1800
8 Ferry Alley
839 Charnelton St
1410 Orchard St
2095 Garden Ave
The Meridian Building
1001 W 1st Ave
990 W 1st Ave
1997 Amazon Pkwy
524 S A St
1856 E 13th Av

East 14th Street 531
Side Pocket Tavern
Moss Street 1741
East 19th Avenue 791
The Vintage
East 18th Avenue 729
Lincoln Street 899
West 11th alley 629
Eugene Municipal Court
East 16th Avenue 403
Lawrence Street 1338
Fillmore Street 2643
East 21st Avenue 699
Lawrence Street 1309
Alder Street 1519
Ferry Street 1650
East 16th Avenue 799
Willamette Alley 1299
Cafe Carpe Diem
East 11th Avenue 743
High Street 1556
Oak Street 1765
East 12th Avenue 417
West 22nd Avenue 1784
Turf Field 4
Spyglass Drive 893
East 12th Avenue 499
Kincaid Street 2585
East 14th Alley 798
East 12th Avenue 595
Washington Federal
Great Blue Heron
West 11th alley 639
East 18th Avenue 770
East 19th Avenue 100
West 11th alley 624
The Taste of India
Charnelton Street 1973
Floral Hill Drive
Yogurt Extreme
West 5th Avenue
Mill Alley 1360
Lawrence Street 163
OR 99
Chao Pra Ya Resturant
East 13th Alley 587
Friendly Alley 2407
East 12th Avenue 593
West 17th Avenue 40
Photonlight.com
West 17th Avenue 44
Ferry Street 1359
East 24t

Mill Street 1681
DVLPMNT
Ferry Street 1631
First Interstate Bank
East 13th Alley 538
East 13th Alley 534
East 13th Alley 544
East 13th Alley 418
East 13th Alley 546
West 11th Avenue 432
West 16th Avenue 9
Wildcraft Cider Works
East 14th Street 501
Valley River Way 1066
East 24th Avenue 525
Taste of India
Madison Street 114
West 1st Avenue 716
West 1st Avenue 708
East 21st Avenue 2085
Ferry Street 851
Garden Way Retirement Community
West 10th Avenue 98
East 20th Avenue 1421
East 12th Avenue 537
West 19th Avenue 15
East 24th Avenue 511
East 24th Avenue 509
Togo's
East 18th Avenue 438
Ferry Street 1800
East 13th Alley 433
Kincaid Street 1251
West 8th Avenue 491
Ferry Street 1477
East 20th Avenue 1417
Kincaid Street 1282
Ferry Street 1801
West 2nd Avenue 499
East 20th Avenue 1409
East 14th Street 332
Patterson Alley 1111
Suds 'em Yourself
Ferry Street 1825
Ferry Street 1843
Ferry Street 1803
Ferry Street 1890
First Place Family Center
Lawrence Street 257
East 20th Avenue 1441
East 14th All

2565 Onyx St
East 20th Avenue 674
East 20th Avenue 651
602 E 8th Ave
877 W Broadway
1733 Mill Alley
1110 W 8th Ave
2500 W 18th Ave
631 E 19th Ave Bldg B
696 W 6th Ave
715 E 16th Ave
590 Country Club Pkwy
1491 Pearl St
572 Polk St
325 E 11th Alley
2727 Alder St
90071 Armitage Rd
2440 Kincaid St
577 W 10th Ave
70 W 29th Ave
2665 Onyx St
885 Van Buren St
931 E 22nd Ave
2819 Alder St
115 W 18th Ave
225 E 19th Ave
1845 Villard St
305 W 7th Ave
1069 W 4th Ave
W/S of Commons N of Kinsrow
2084 Willamette St
66 Club Rd #390
661 E 18th Ave
2191 Kincaid St
1259 Monroe St
1305 W 17th Ave
475 W 7th Ave
1060 W 11th Ave
338 W 11th Ave #108
2565 Van Buren St
2505 Alder St
1755 W 17th Ave
1797 Orchard St
92 Grand St
236 N Grand St
495 W 7th Ave Ste. 300
894 E 21st Ave
1157 High St
115 W 6th Ave
330 S Garden Way Suite 100
275 E 36th Ave
313 E 8th Ave
1288 W 8th Ave
1602 Villard St
687 Cheshire Ave
1200 Executive Pkwy Suite 320
2628 Alder St
68 W 29th Ave
1757 Mill Alley
310 W 19th Ave
250 Cheshire Ave
4

2010 Emerald St
190 E 18th Ave
2406 Agate St
227 W 5th Ave
1932 Columbia St
1603 E 22nd Ave
2777 Friendly St
2790 Potter St
528 Tyler St
1355 Oak St
1692 Jefferson St
257 1/2 W 15th Ave
282 W 23rd Ave
W/S of VR Way S of VR Inn
745 E 20th Ave
1510 Riverview St
1881 Garfield St
2711 Kincaid St
1601 Chambers St
2859 Capital Dr
1228 Charnelton Alley
Morse Events Center
2860 Mill St
622 E 22nd Ave C
2295 Chambers St
1745 W 5th Ave # J
2303 McMorran St
2585 Agate St
88 Jackson St
3460 Ferry St
2895 Kincaid St
745 W 10th Ave
948 Lariat Dr
1327 Lawrence St
1745 Olive St
1191 High St
210 Crest Dr
2265 Rose Ln
618 E 19th Ave
2300 Lincoln St
Hwy 99 Path
34901 Frank Parrish Rd
W/S of River Rd S of Briarcliff
E/S of Chambers N of 5th
1175 W 15th Ave
770 E 23rd Ave
Home Depot
431 W 13th Ave
3815 W 11th Ave #100
1593 Jefferson St
950 Tyinn St
905 W 7th Ave
3435 McKenna Dr
2324 Alder St
1190 W 7th Ave
2530 Fairmount Blvd
272 W 11th Ave
1010c Green Acres Rd
S/S of 18th E of Chambers
E/S of Goodpstre Is

1011 Valley River Way #119
1887 W 9th Pl
264 High St
1649 E 23rd Ave
1270 E 22nd Ave
3123 Kinsrow Ave
1997 Garden Ave
1060 E 28th Ave
715 W 4th Ave
3515 Kinsrow Ave
641 1/2 Madison St
5 High Building
2040 Alder St
622 E 17th Ave
40 E 30th Ave
254 W 19th Ave
732 W 6th Ave
935 Oak St
144 W 19th Ave
167 N Adams St
427 W 12th Alley
1544 Patterson St
1075 International Way
1706 Jefferson St
107 Van Buren St
2475 Van Ness St
772 Olive St
1725 Coburg Rd
210 N Adams St
Monroe/6th Station Inbound
2511 W 11th Ave
57 Lawrence St
1369 W 6th Ave #280
1954 Monroe St
1410 W 6th Ave
1177 Rio Glen Dr
1697 Augusta St
3 Patterson Alley
2741 Shadowview
S/S of Wagner E of Cubit
184 Commons Dr
1112 E 25th Ave
2417 Agate St
1978 E 19th Ave
1900 Chambers St
497 Oakway Center
346 W 16th Ave
71 N Jackson St
1443 Harbor Dr
1142 Willagillespie Rd # 28
2890 Potter St
1080 W 7th Ave
1708 Columbia St
3007 W 11th Ave
1385 Oakway Rd
334 High St
2330 Willamette St
115 W Broadway
1601 Oak St #1B
1770 W 11th Ave
941 Pear

In [16]:
trips_selected = trips[trips.Location.isin(tripLocSelected)]

In [17]:
originCounts = trips_selected[trips_selected.OriginDestination == 'Origin'][['Location', 'RouteID', 'UserID']].groupby('Location').agg({"RouteID": lambda x: x.nunique(), "UserID": lambda x: x.nunique()})

In [18]:
originCounts.head()

,RouteID,UserID
Location,,
10th & Olive NE,313,146
10th & Olive SE,9776,2039
10th & Pearl,3307,1170
12th & Willamette,17766,2560
13th & Kincaid,42107,7020


In [19]:
len(trips_selected.UserID.unique())

29071

In [20]:
trips_selected.shape[0]

783901

In [21]:
data = {'name':originCounts.index.values,
        'ntrips':originCounts.RouteID.values,
        'nbikers':originCounts.UserID.values}

In [22]:
df = pd.DataFrame(data)

In [23]:
origin_df = stations.merge(df, how='inner', on='name')

In [24]:
origin_df.head()

,station id,name,geometry,ntrips,nbikers
0,hub_3828,Eugene Family YMCA,POINT (-13701591.712 5471253.204),11175,1567
1,hub_3829,Monroe St & Blair Blvd,POINT (-13704001.270 5473411.225),5785,1254
2,hub_3830,10th & Olive SE,POINT (-13702770.365 5472975.425),9776,2039
3,hub_3831,16th & Willamette,POINT (-13702642.423 5471973.384),6346,1359
4,hub_3832,13th & Pearl @ Pearl side,POINT (-13702278.750 5472449.473),7617,2207


In [25]:
origin_df.to_file(r'T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\OriginCounts.shp')

In [26]:
destinationCounts = trips_selected[trips_selected.OriginDestination == 'Destination'][['Location', 'RouteID', 'UserID']].groupby('Location').agg({"RouteID": lambda x: x.nunique(), "UserID": lambda x: x.nunique()})

In [27]:
data = {'name':destinationCounts.index.values,
        'ntrips':destinationCounts.RouteID.values,
        'nbikers':destinationCounts.UserID.values}
df = pd.DataFrame(data)
destination_df = stations.merge(df, how='inner', on='name')
destination_df.to_file(r'T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\DestinationCounts.shp')